# Read the dataset

In [1]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt #描画ライブラリ
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
# import pathlib
# import glob
# import math

# Show all the rows and columns up to 200
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# Data set for seller/buyer
df_prediction_afterDA = pd.read_csv('df_prediction.csv', sep=',', header=0)
df_prediction_afterDA = df_prediction_afterDA[["DateTime", "Spot", "High", "Low", "Close", "Close_pred"]]
df_prediction_afterDA.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df_prediction.csv'

In [ ]:
# Data set for trader
df_prediction_beforeDA = pd.read_csv('df_prediction_trader.csv', sep=',', header=0)
df_prediction_beforeDA.head()

# Benchmark

## Benchmark 1 (Trade only on DA market)
- No strategy: Players trade on DA market as much as possible to avoid imbalance risks (100% position only on DA market)<p>
- Calculate return and risk with "Spot price"

### Requierd dataset

In [ ]:
# Need actual spot, high, close price
df_benchmark1 = df_prediction_afterDA.copy()

# Drop Close_pred and others for intraday market since players who trade without strategy need only spot price.
df_benchmark1 = df_benchmark1.drop(["Close_pred", "High", "Low", "Close"], axis=1)

df_benchmark1.head()

### Evaluation

In [ ]:
# fig, ax = plt.subplots(1, figsize=plt.figaspect(.25))
plt.scatter(x=df_benchmark1["Spot"].std(), y=df_benchmark1["Spot"].mean(), color="r", label="100% Spot price")
print("Expected return: {}".format(df_benchmark1["Spot"].mean().round(2)))
print("Standard deviation: {}".format(df_benchmark1["Spot"].std().round(2)))
print("Sharp ratio: {}".format(round(df_benchmark1["Spot"].mean()/df_benchmark1["Spot"].std(), 2)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.legend();

In [ ]:
df_benchmark1["Spot"].hist(bins=100);

## Benchmark2 (Trade only on Intraday market)
- No strategy: Players trade on Intra market (100% position only on Intra market)<p>
    --> ※ This strategy would be not realistic due to imbalence risk, but just try to look at the performance <p>
- Calculate return and risk with "Close price"

### Required dataset

In [ ]:
# Need actual spot, high, close price
df_benchmark2 = df_prediction_afterDA.copy()

# Drop Close_pred and others for intraday market since players who trade without strategy need only spot price.
df_benchmark2 = df_benchmark2.drop(["Close_pred", "High", "Low", "Spot"], axis=1)

df_benchmark2.head()

### Evaluation

In [ ]:
# fig, ax = plt.subplots(1, figsize=plt.figaspect(.25))
plt.scatter(x=df_benchmark2["Close"].std(), y=df_benchmark2["Close"].mean(), color="r", label="100% Close price")
print("Expected return: {}".format(df_benchmark2["Close"].mean().round(2)))
print("Standard deviation: {}".format(df_benchmark2["Close"].std().round(2)))
print("Sharp ratio: {}".format(round(df_benchmark2["Close"].mean()/df_benchmark2["Close"].std(), 2)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.legend();

In [ ]:
df_benchmark2["Close"].hist(bins=100);

As a result, both expected return and standard deviation on Intraday market are higher than that on DA market <p>
--> Sharp ratio on Intraday market in lower than that of DA market <p>
--> Return is high, but TOO RISKY

※ The benchmark1 will be compared with the following strategies. 
- Strategies for seller aim to sell their posision for higher price than the benchmark
- Strategies for buyer aim to buy their posision for lower price than the benchmark

# Trading strategies for sellers (e.g. power producers)

[Assumption]<p>
1)	All of the market participants are price-takers and their trading decisions do not affect the price on any market. <p>
2)	Auction strategy at the DA market is ignored. Realised spot price is used for evaluating the portfolios since spot price is determined based on a single price auction system.<p>
3)	Strategic decisions are made based only on the price before each prediction point, and trading performance is evaluated based on realised price after the market.<p>
4)	Trading volume is ignored. Therefore, transaction fees are also ignored. Return and risk will be defined in the next section.<p>
5)	The iceberg order  is ignored since the information of the ask/bid order book is not available. <p>
6)	All the orders can be executed as far as the price of the order is within the range of the high and low price of that day. It also means there are no penalties due to imbalance.<p>
7)	For electricity producers, the influence of curtailment on their operating and trading decision making is ignored. Curtailment is restriction of generating electricity especially for renewable energy power plants.<p>
8)	For retailers, the influence of supply contracts with customers on trading decision making is ignored.<p>

## Strategy 1 (Trading with prediction before DA) without execution strategy
- Choose the higher price of　1day-lagged spot price or the predicted close price(before DA)<p>
    --> Determinint trading position depends on the prices

In [ ]:
Benchmark_seller = df_benchmark1["Spot"].round(2)
Benchmark_seller

### Required dataset

In [ ]:
# Need predicted price before DA market and 1 dayahead spot price
df_seller_strategy1 = df_prediction_beforeDA.copy()
df_seller_strategy1["Spot_1dayahead"] = df_seller_strategy1["Spot"].shift(48)
# Drop rows that include NaN
df_seller_strategy1 = df_seller_strategy1.dropna(how='any', axis=0).reset_index(drop=True)

# Make lists for price information
SpotLag_list = list(df_seller_strategy1["Spot_1dayahead"])
Spot_list = list(df_seller_strategy1["Spot"])
High_list = list(df_seller_strategy1["High"])
Close_list = list(df_seller_strategy1["Close"])
Pred_list = list(df_seller_strategy1["Close_pred"])
# Judge_success = []

# list for executed orders
Executed = []
Judge = []
Position = []
for sl, s, h, c, p  in zip(SpotLag_list, Spot_list, High_list, Close_list, Pred_list):
    # Trade on DA market
    if sl >= p:
        Executed.append(s)
        Judge.append("True")
        Position.append("DA")
    # Trade on Intra markets
    else:
        if p <= h:
            Executed.append(p)
            Judge.append("True")
            Position.append("Intra")
        else:
            Executed.append(c)
            Judge.append("False")        
            Position.append("Intra")
df_seller_strategy1["ExecutedOrder"] = pd.Series(Executed)
df_seller_strategy1["Judge"] = pd.Series(Judge)
df_seller_strategy1["Position"] = pd.Series(Position)

#     Judge_success.append((df_seller_bench["Judge_" + str(i) + "%"] == 'True').sum())
# #For graph
# Judge_success = pd.Series(Judge_success)

In [ ]:
df_seller_strategy1.tail()

### Evaluation

In [ ]:
# The percentage of DA position 
DA_position = df_seller_strategy1["Position"].value_counts()[0] 
Intra_position = df_seller_strategy1["Position"].value_counts()[1]
DA_ratio = round(DA_position / (DA_position+Intra_position)*100, 2)
DA_ratio

In [ ]:
# fig, ax = plt.subplots(1, figsize=plt.figaspect(.25))
plt.scatter(x=df_seller_strategy1["ExecutedOrder"].std(), y=df_seller_strategy1["ExecutedOrder"].mean(), color="r", label="Combination of DA and Intraday")
print("Position: DA {}".format(round(DA_position / (DA_position+Intra_position)*100, 2)) + "%")
print("Expected return: {}".format(df_seller_strategy1["ExecutedOrder"].mean().round(2)))
print("Standard deviation: {}".format(df_seller_strategy1["ExecutedOrder"].std().round(2)))
print("Sharp ratio: {}".format(round(df_seller_strategy1["ExecutedOrder"].mean()/df_seller_strategy1["ExecutedOrder"].std(), 2)))
print("Information ratio: {}".format(round((df_seller_strategy1["ExecutedOrder"] - Benchmark_seller).mean()/(df_seller_strategy1["ExecutedOrder"] - Benchmark_seller).std(), 2)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.legend();

In [ ]:
df_seller_strategy1["ExecutedOrder"].hist(bins=100);

Much higher than benchmarks

## Strategy 1 with execusion strategy
- Derive strategy1 with execution strategy <p>
※ 10-fold evaluation is necessary

### Required dataset

In [ ]:
# Need predicted price before DA market and 1 dayahead spot price
df_seller_strategy1_ex = df_prediction_beforeDA.copy()
df_seller_strategy1_ex["Spot_1dayahead"] = df_seller_strategy1_ex["Spot"].shift(48)
# Drop rows that include NaN
df_seller_strategy1_ex = df_seller_strategy1_ex.dropna(how='any', axis=0).reset_index(drop=True)

# Make lists for price information
SpotLag_list = list(df_seller_strategy1_ex["Spot_1dayahead"])
Spot_list = list(df_seller_strategy1["Spot"])
High_list = list(df_seller_strategy1_ex["High"])
Close_list = list(df_seller_strategy1_ex["Close"])
# Judge_success = []

# Order and execution with execusion range
for i in list(range(0, 51, 1)):
    # list for executed orders
    Executed_i = []
    # list for checking the execution results
    Judge_i = []    
    Position_i = []    
    # Derive the predicted price with the range from +0% ~ +20%
    Pred_list_i = list((df_seller_strategy1_ex["Close_pred"]* (1 + i/100)).round(2))
    for sl, s, h, c, p in zip(SpotLag_list, Spot_list, High_list, Close_list, Pred_list_i):
        # Trade on DA market
        if sl >= p:
            Executed_i.append(s)
            Judge.append("True")
            Position_i.append("DA")
        # Trade on Intra markets
        else:
            if p <= h:
                Executed_i.append(p)
                Judge_i.append("True")
                Position_i.append("Intra")
            else:
                Executed_i.append(c)
                Judge_i.append("False")
                Position_i.append("Intra")
    df_seller_strategy1_ex["Exec_" + str(i) + "%"] = pd.Series(Executed_i)
    df_seller_strategy1_ex["Judge_" + str(i) + "%"] = pd.Series(Judge_i)
    df_seller_strategy1_ex["Position_" + str(i) + "%"] = pd.Series(Position_i)

#     Judge_success.append((df_seller_bench["Judge_" + str(i) + "%"] == 'True').sum())
# #For graph
# Judge_success = pd.Series(Judge_success)

In [ ]:
df_seller_strategy1_ex.head()

### Evaluation

- Seek the best execution point

In [ ]:
n_splits=100

train_size = df_seller_strategy1_ex.index[-1]
sample_size = int(train_size/n_splits)
train_index_list = list(np.linspace(sample_size, train_size, n_splits, endpoint = True, dtype='int'))
train_index_list

# # Confirming the split logic
# for train_index in train_index_list:
#     # Divide the train/valid set  into 10 folds and pick up it.
#     X_train = df_seller_strategy1_ex.iloc[:train_index]
#     print("start:", train_index - sample_size)
#     print("end:", train_index)

In [ ]:
BestExec = []

for train_index in train_index_list:
    X_train = df_seller_strategy1_ex[:train_index]

    cols = []

    # List for the results for evaluation
    PortfolioReturn = []
    StandardDeviation = []
    Max = []
    Min = []
    SharpRatio = []
    InformationRatio = []
    
    for i in list(range(0, 51, 1)):
        Return_i = []
        Exec_list_i = list(X_train["Exec_" + str(i) + "%"]) 
        for e in Exec_list_i:
            Return_i.append(e)

        Return_i = pd.Series(Return_i)
        PortfolioReturn.append(Return_i.mean().round(2))
        StandardDeviation.append(Return_i.std().round(2))
        Max.append(Return_i.max().round(2))
        Min.append(Return_i.min().round(2))
        SharpRatio.append((Return_i.mean()/Return_i.std()).round(2))
        InformationRatio.append(round((Return_i - Benchmark_seller).mean()/(Return_i - Benchmark_seller).std(), 3))

        # Make columns names
        cols.append(i)

    # Make dataframe for evaluation and  switch columns and row.
    df_seller_strategy1_ex_eval = pd.DataFrame()
    # df_seller_strategy1_ex_eval = df_seller_strategy1_ex_eval.T

    # Add columns for evaluation metrics
    df_seller_strategy1_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
    df_seller_strategy1_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
    df_seller_strategy1_ex_eval["Max"] = pd.Series(Max)
    df_seller_strategy1_ex_eval["Min"] = pd.Series(Min)
    df_seller_strategy1_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
    df_seller_strategy1_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

    df_seller_strategy1_ex_eval.index = cols
    df_seller_strategy1_ex_eval =  df_seller_strategy1_ex_eval.reset_index()
    df_seller_strategy1_ex_eval = df_seller_strategy1_ex_eval.rename(columns={"index": "ExecBuffer(%)"})

    Exec = df_seller_strategy1_ex_eval[df_seller_strategy1_ex_eval["InformationRatio"] == df_seller_strategy1_ex_eval["InformationRatio"].max()]
    Exec = Exec["ExecBuffer(%)"][Exec["StandardDeviation"] == Exec["StandardDeviation"].min()].iloc[0]
    BestExec.append(Exec)

In [ ]:
# The best execution of 100 different periods
BestExec = pd.Series(BestExec)
plt.xlabel('Number of backtest')
plt.ylabel('Execution adjustment (%)')
BestExec.plot();

In [ ]:
Best_i = BestExec.iloc[-1]
Best_i

- Plot all the possible portfolios

In [ ]:
cols = []

# List for the results for evaluation
PortfolioReturn = []
StandardDeviation = []
Max = []
Min = []
SharpRatio = []
InformationRatio = []

for i in list(range(0, 51, 1)):
    Return_i = []
    Exec_list_i = list(df_seller_strategy1_ex["Exec_" + str(i) + "%"]) 
    for e in Exec_list_i:
        Return_i.append(e)

    Return_i = pd.Series(Return_i)
    PortfolioReturn.append(Return_i.mean().round(2))
    StandardDeviation.append(Return_i.std().round(2))
    Max.append(Return_i.max().round(2))
    Min.append(Return_i.min().round(2))
    SharpRatio.append((Return_i.mean()/Return_i.std()).round(2))
    InformationRatio.append(round((Return_i - Benchmark_seller).mean()/(Return_i - Benchmark_seller).std(), 3))

    # Make columns names
    cols.append("Exec+" + str(i) + "%")

# Make dataframe for evaluation and  switch columns and row.
df_seller_strategy1_ex_eval = pd.DataFrame()

#各算出結果をdf_portfolio_benchの列へ追加する
df_seller_strategy1_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_seller_strategy1_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_seller_strategy1_ex_eval["Max"] = pd.Series(Max)
df_seller_strategy1_ex_eval["Min"] = pd.Series(Min)
df_seller_strategy1_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
df_seller_strategy1_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

df_seller_strategy1_ex_eval.index = cols

In [ ]:
df_seller_strategy1_ex_eval.head()

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_seller_strategy1_ex_eval[df_seller_strategy1_ex_eval["StandardDeviation"] == df_seller_strategy1_ex_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].max()].reset_index()
GMVP

In [ ]:
Best = df_seller_strategy1_ex_eval[df_seller_strategy1_ex_eval["InformationRatio"] == df_seller_strategy1_ex_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_seller_strategy1_ex_eval["StandardDeviation"], df_seller_strategy1_ex_eval["PortfolioReturn"], c=df_seller_strategy1_ex_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')

# Global minimum variance portfolio
i = 17
DA_position_i = df_seller_strategy1_ex["Position_" + str(i) + "%"].value_counts()[0] 
Intra_position_i = df_seller_strategy1_ex["Position_" + str(i) + "%"].value_counts()[1]

plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio]")
print("Position: DA {}".format(round(DA_position_i / (DA_position_i+Intra_position_i)*100, 2)) + "%")
print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Sharp ratio: {}".format(GMVP["SharpRatio"][0]))
print("Information ratio: {}".format(GMVP["InformationRatio"][0]))

# The Best Execusion
i = Best_i
DA_position_i = df_seller_strategy1_ex["Position_" + str(i) + "%"].value_counts()[0] 
Intra_position_i = df_seller_strategy1_ex["Position_" + str(i) + "%"].value_counts()[1]

plt.plot(df_seller_strategy1_ex["Exec_" + str(i) + "%"].std(), df_seller_strategy1_ex["Exec_" + str(i) + "%"].mean(), "b*", markersize=15.0, label="Best Execution")
print("[Best Execution Portfolio]")
print("Position: DA {}".format(round(DA_position_i / (DA_position_i+Intra_position_i)*100, 2)) + "%")
print("Expected return: {}".format(df_seller_strategy1_ex["Exec_" + str(i) + "%"].mean().round(2)))
print("Standard deviation: {}".format(df_seller_strategy1_ex["Exec_" + str(i) + "%"].std().round(2)))
print("Sharp ratio: {}".format(round(df_seller_strategy1_ex["Exec_" + str(i) + "%"].mean() / df_seller_strategy1_ex["Exec_" + str(i) + "%"].std(),2)))
print("Information ratio: {}".format(round((df_seller_strategy1_ex["Exec_" + str(i) + "%"] - Benchmark_seller).mean() / (df_seller_strategy1_ex["Exec_" + str(i) + "%"] - Benchmark_seller).std(),2)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Information Ratio')
plt.legend(loc="upper left");

## Strategy 2 (Trading with prediction after DA)
--> Players have the information of spot price and predicted price after DA market <p>
1) Players trade with pre-determined portfolio proportion that ranges from 0% to 100%. <p>
2) Positions are derived with execution range if players have the possition of intraday market <p>

### Required dataset

In [ ]:
# Need predicted price after DA market and spot price
df_seller_strategy2 = df_prediction_afterDA.copy()

# Make lists for price information
Spot_list = list(df_seller_strategy2["Spot"])
High_list = list(df_seller_strategy2["High"])
Close_list = list(df_seller_strategy2["Close"])
Pred_list_i = list(df_seller_strategy2["Close_pred"])
# Judge_success = []
                   
# list for executed orders
Executed_i = []
# list for checking the execution results
Judge_i = []    
                   
# Calculate executed price
for h, c, p in zip(High_list, Close_list, Pred_list_i):     
    # Trade on Intra markets
    if p <= h:
        Executed_i.append(p)
        Judge_i.append("True")
    else:
        Executed_i.append(c)
        Judge_i.append("False")                 

df_seller_strategy2["ExecutedOrder"] = pd.Series(Executed_i)
df_seller_strategy2["Judge"] = pd.Series(Judge_i)

df_seller_strategy2.tail()

### Evaluation

In [ ]:
portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

cols = []

Return = [] 
PortfolioReturn = []
StandardDeviation = []
SharpRatio = []
InformationRatio = []

Spot_list = list(df_seller_strategy2["Spot"])
Exec_list = list(df_seller_strategy2["ExecutedOrder"])

for weight in portfolio_weights:
    Return = []
    for spot, intra in zip(Spot_list, Exec_list):
        # DA market 100%
        if weight == 1.0:
            Return.append(spot*weight)
        # Trade both on DA and Intra day
        else:
            Return.append(spot*weight + intra*(1-weight))
    # Set colmun name
    cols.append("DA" + str(weight*100) + "%")

    # Results
    Return = pd.Series(Return)
    PortfolioReturn.append(Return.mean().round(2))
    StandardDeviation.append(Return.std().round(2))
    SharpRatio.append((Return.mean()/Return.std()).round(2))
    InformationRatio.append(round((Return - Benchmark_seller).mean() / (Return - Benchmark_seller).std(), 3))
    
# Transpose columns and rows
df_seller_strategy2_eval = pd.DataFrame()
df_seller_strategy2_eval = df_seller_strategy2_eval.T

# Add columns for evaluation metrics
df_seller_strategy2_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_seller_strategy2_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_seller_strategy2_eval["Max"] = pd.Series(Max)
df_seller_strategy2_eval["Min"] = pd.Series(Min)
df_seller_strategy2_eval["SharpRatio"] = pd.Series(SharpRatio)
df_seller_strategy2_eval["InformationRatio"] = pd.Series(InformationRatio)
df_seller_strategy2_eval.index = cols

In [ ]:
df_seller_strategy2_eval

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_seller_strategy2_eval[df_seller_strategy2_eval["StandardDeviation"] == df_seller_strategy2_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].max()].reset_index()
GMVP

In [ ]:
# Pick up the best portrolio
Best = df_seller_strategy2_eval[df_seller_strategy2_eval["InformationRatio"] == df_seller_strategy2_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_seller_strategy2_eval["StandardDeviation"], df_seller_strategy2_eval["PortfolioReturn"], c=df_seller_strategy2_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')

# Global minimum variance portfolio
plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio]")
print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Sharp ratio: {}".format(GMVP["SharpRatio"][0]))
print("Information ratio: {}".format(GMVP["InformationRatio"][0]))

# The Best Execusion
plt.plot(Best["StandardDeviation"], Best["PortfolioReturn"], "b*", markersize=15.0, label="Best Execution")
print("[Best Execution Portfolio]")
print("Expected return: {}".format(Best["PortfolioReturn"][0]))
print("Standard deviation: {}".format(Best["StandardDeviation"][0]))
print("Sharp ratio: {}".format(Best["SharpRatio"][0]))
print("Information ratio: {}".format(Best["InformationRatio"][0]))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Information Ratio')
plt.legend(loc="upper left");

## Strategy 2 with execusion strategy
--> Players have the information of spot price and predicted price after DA market <p>
1) Players trade with pre-determined portfolio proportion that ranges from 0% to 100%. <p>
2) Positions is derived with execution range if players have the possition of intraday market <p>

### Required dataset

In [ ]:
# Need predicted price after DA market and spot price
df_seller_strategy2_ex = df_prediction_beforeDA.copy()

# Make lists for price information
Spot_list = list(df_seller_strategy2_ex["Spot"])
High_list = list(df_seller_strategy2_ex["High"])
Close_list = list(df_seller_strategy2_ex["Close"])

# Order and execution with execusion range
for i in list(range(0, 51, 1)):
    # list for executed orders
    Executed_i = []
    # list for checking the execution results
    Judge_i = []    
    # Derive the predicted price with the range
    Pred_list_i = list((df_seller_strategy2_ex["Close_pred"]* (1 + i/100)).round(2))
    for h, c, p in zip(High_list, Close_list, Pred_list_i):     
        # Trade on Intra markets
        if p <= h:
            Executed_i.append(p)
            Judge_i.append("True")
        else:
            Executed_i.append(c)
            Judge_i.append("False")                 
                
    df_seller_strategy2_ex["Exec_" + str(i) + "%"] = pd.Series(Executed_i)
    df_seller_strategy2_ex["Judge_" + str(i) + "%"] = pd.Series(Judge_i)

In [ ]:
df_seller_strategy2_ex.tail()

### Evaluation

- Look for the best execution

In [ ]:
n_splits=100

train_size = df_seller_strategy2_ex.index[-1]
sample_size = int(train_size/n_splits)
train_index_list = list(np.linspace(sample_size, train_size, n_splits, endpoint = True, dtype='int'))
train_index_list

# # Confirming the split logic
# for train_index in train_index_list:
#     # Divide the train/valid set  into 10 folds and pick up it.
#     X_train = df_seller_strategy2_ex.iloc[:train_index]
#     print("start:", train_index - sample_size)
#     print("end:", train_index)

In [ ]:
# This code is used only for confirming the best execution point
BestExec = []

for train_index in train_index_list:
    X_train = df_seller_strategy2_ex[:train_index]
    Spot_list = list(X_train["Spot"])
   
    cols = []
    PortfolioReturn = []
    StandardDeviation = []
    SharpRatio = []
    InformationRatio = []
    
    portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    
    for i in list(range(0, 51, 1)):
        for weight in portfolio_weights:
            Return_iw = []
            Exec_list = list(X_train["Exec_" + str(i) + "%"])
            for spot, intra in zip(Spot_list, Exec_list):
                # DA market 100%
                if weight == 1.0:
                    Return_iw.append(spot*weight)
                # Trade both on DA and Intra day
                else:
                    Return_iw.append(spot*weight + intra*(1-weight))
            # Set colmun name
            if weight == 1.0:
                cols.append(str(0))
            else:
                cols.append(str(i))

            # Results
            Return_iw = pd.Series(Return_iw)
            PortfolioReturn.append(Return_iw.mean().round(2))
            StandardDeviation.append(Return_iw.std().round(2))
            SharpRatio.append((Return_iw.mean()/Return_iw.std()).round(2))
            InformationRatio.append(round((Return_iw - Benchmark_seller).mean() / (Return_iw - Benchmark_seller).std(), 3))

    # Results of every portfolios derived by execution strateties
    df_seller_strategy2_ex_eval = pd.DataFrame()
    df_seller_strategy2_ex_eval = df_seller_strategy2_ex_eval.T

    # Add columns for evaluation metrics
    df_seller_strategy2_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
    df_seller_strategy2_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
    df_seller_strategy2_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
    df_seller_strategy2_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

    df_seller_strategy2_ex_eval.index = cols
    df_seller_strategy2_ex_eval = df_seller_strategy2_ex_eval.reset_index()
    df_seller_strategy2_ex_eval = df_seller_strategy2_ex_eval.rename(columns={"index": "ExecBuffer(%)"})

    Exec = df_seller_strategy2_ex_eval[df_seller_strategy2_ex_eval["InformationRatio"] == df_seller_strategy2_ex_eval["InformationRatio"].max()]
    Exec = Exec["ExecBuffer(%)"][Exec["StandardDeviation"] == Exec["StandardDeviation"].min()].iloc[0]
    BestExec.append(Exec)

In [ ]:
BestExec = pd.Series(BestExec).astype(int)
plt.xlabel('Number of backtest')
plt.ylabel('Execution adjustment (%)')
BestExec.plot();

In [ ]:
Best_i = BestExec.iloc[-1]
Best_i

- Plot all the possible portfolios

In [ ]:
Spot_list = list(df_seller_strategy2_ex["Spot"])

portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
execution_buffers = list(range(0, 51, 1))

cols = []

Return = [] 
PortfolioReturn = []
StandardDeviation = []
SharpRatio = []
InformationRatio = []

for i in execution_buffers:
    Exec_list = list(df_seller_strategy2_ex["Exec_" + str(i) + "%"])
    for weight in portfolio_weights:
        Return_iw = []
        for spot, intra in zip(Spot_list, Exec_list):
            # DA market 100%
            if weight == 1.0:
                Return_iw.append(spot*weight)
            # Trade both on DA and Intra day
            else:
                Return_iw.append(spot*weight + intra*(1-weight))
        # Set colmun name for execution buffer
        if weight == 1.0:
            cols.append("DA100%")
        else:
            cols.append("DA" + str(weight*100) + "%_Exec+" + str(i) + "%")

        # Results
        Return_iw = pd.Series(Return_iw)
        PortfolioReturn.append(Return_iw.mean().round(2))
        StandardDeviation.append(Return_iw.std().round(2))
        SharpRatio.append((Return_iw.mean()/Return_iw.std()).round(2))
        InformationRatio.append(round((Return_iw - Benchmark_seller).mean() / (Return_iw - Benchmark_seller).std(), 3))

# Transpose columns and rows
df_seller_strategy2_ex_eval = pd.DataFrame()
df_seller_strategy2_ex_eval = df_seller_strategy2_ex_eval.T

# Add columns for evaluation metrics
df_seller_strategy2_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_seller_strategy2_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_seller_strategy2_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
df_seller_strategy2_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

df_seller_strategy2_ex_eval.index = cols

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_seller_strategy2_ex_eval[df_seller_strategy2_ex_eval["StandardDeviation"] == df_seller_strategy2_ex_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].max()].reset_index()
GMVP

In [ ]:
# Pick up the best portfolio
Best = df_seller_strategy2_ex_eval[df_seller_strategy2_ex_eval["InformationRatio"] == df_seller_strategy2_ex_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_seller_strategy2_ex_eval["StandardDeviation"], df_seller_strategy2_ex_eval["PortfolioReturn"], c=df_seller_strategy2_ex_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')

# Global minimum variance portfolio
plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio: " + str(GMVP.index[0]) + "]")
print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Sharp ratio: {}".format(GMVP["SharpRatio"][0]))
print("Information ratio: {}".format(GMVP["InformationRatio"][0]))

# The Best Execusion
DA_weight = 0.5
i = Best_i
Best_Portfolio = (DA_weight * df_seller_strategy2_ex["Spot"]) + ((1 - DA_weight) * df_seller_strategy2_ex["Exec_" + str(i) + "%"])
plt.plot(Best_Portfolio.std(), Best_Portfolio.mean(), "b*", markersize=15.0, label="Best Execution")
print("[Best Execution Portfolio: " + str(DA_weight) + "%_Exec+" + str(i) + "%]")
print("Expected return: {}".format(Best_Portfolio.mean().round(2)))
print("Standard deviation: {}".format(Best_Portfolio.std().round(2)))
print("Sharp ratio: {}".format(round(Best_Portfolio.mean() / Best_Portfolio.std(), 2)))
print("Information ratio: {}".format(round((Best_Portfolio - Benchmark_seller).mean() / (Best_Portfolio - Benchmark_seller).std(), 3)))

plt.title('Strategic portfolios with excution adjustment for sellers')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Information ratio')
plt.legend(loc="upper right");

# Trading strategy for buyers (e.g. retailers)
- Basically the same as the strategies for seller
- Buyer aims to buy for cheaper price

In [ ]:
# Benchmark for buyer's strategy
Benchmark_buyer = -df_benchmark1["Spot"].round(2)
Benchmark_buyer

## Strategy 1 (Trading with prediction before DA) without execution strategy

### Required dataset

In [ ]:
# Need predicted price before DA market and 1 dayahead spot price
df_buyer_strategy1 = df_prediction_beforeDA.copy()
df_buyer_strategy1["Spot_1dayahead"] = df_buyer_strategy1["Spot"].shift(48)
# Drop rows that include NaN
df_buyer_strategy1 = df_buyer_strategy1.dropna(how='any', axis=0).reset_index(drop=True)

# Make lists for price information
SpotLag_list= list(df_buyer_strategy1["Spot_1dayahead"])
Spot_list = list(df_buyer_strategy1["Spot"])
Low_list = list(df_buyer_strategy1["Low"])
Close_list = list(df_buyer_strategy1["Close"])
Pred_list = list(df_buyer_strategy1["Close_pred"])
# Judge_success = []

# list for executed orders
Executed = []
Judge = []
Position = []
for sl, s, l, c, p  in zip(SpotLag_list, Spot_list, Low_list, Close_list, Pred_list):
    # Trade on DA market
    if sl <= p:
        Executed.append(-s)
        Judge.append("True")
        Position.append("DA") 
    # Trade on Intra markets
    else:
        if p <= l:
            Executed.append(-p)
            Judge.append("True")
            Position.append("Intra") 
        else:
            Executed.append(-c)
            Judge.append("False")  
            Position.append("Intra") 

df_buyer_strategy1["ExecutedOrder"] = pd.Series(Executed)
df_buyer_strategy1["Judge"] = pd.Series(Judge)
df_buyer_strategy1["Position"] = pd.Series(Position)

df_buyer_strategy1.tail()

### Evaluation

In [ ]:
# The percentage of DA position
DA_position = df_buyer_strategy1["Position"].value_counts()[0] 
Intra_position = df_buyer_strategy1["Position"].value_counts()[1]
DA_ratio = round(DA_position / (DA_position+Intra_position)*100, 2)
DA_ratio

In [ ]:
# fig, ax = plt.subplots(1, figsize=plt.figaspect(.25))
plt.scatter(x=df_buyer_strategy1["ExecutedOrder"].std(), y=df_buyer_strategy1["ExecutedOrder"].mean(), color="r", label="Combination of DA and Intraday")
print("Position: DA {}".format(round(DA_position / (DA_position+Intra_position)*100, 2)) + "%")
print("Expected return: {}".format(df_buyer_strategy1["ExecutedOrder"].mean().round(2)))
print("Standard deviation: {}".format(df_buyer_strategy1["ExecutedOrder"].std().round(2)))
print("Sharp ratio: {}".format(round(df_buyer_strategy1["ExecutedOrder"].mean()/df_buyer_strategy1["ExecutedOrder"].std(),2)))
print("Information ratio: {}".format(round((df_buyer_strategy1["ExecutedOrder"] - Benchmark_buyer).mean() / (df_buyer_strategy1["ExecutedOrder"] - Benchmark_buyer).std(),3)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.legend();

In [ ]:
df_buyer_strategy1["ExecutedOrder"].hist(bins=100);

## Strategy 1 with execusion strategy

### Required dataset

In [ ]:
# Need predicted price before DA market and 1 dayahead spot price
df_buyer_strategy1_ex = df_prediction_beforeDA.copy()
df_buyer_strategy1_ex["Spot_1dayahead"] = df_buyer_strategy1_ex["Spot"].shift(48)
# Drop rows that include NaN
df_buyer_strategy1_ex = df_buyer_strategy1_ex.dropna(how='any', axis=0).reset_index(drop=True)

# Make lists for price information
SpotLag_list= list(df_buyer_strategy1_ex["Spot_1dayahead"])
Spot_list = list(df_buyer_strategy1_ex["Spot"])
Low_list = list(df_buyer_strategy1_ex["Low"])
Close_list = list(df_buyer_strategy1_ex["Close"])
# Judge_success = []

# Order and execution with execusion range
for i in list(range(0, 51, 1)):
    # list for executed orders
    Executed_i = []
    # list for checking the execution results
    Judge_i = []    
    Position_i = []
    # Derive the predicted price with the range from +0% ~ +20%
    Pred_list_i = list((df_buyer_strategy1_ex["Close_pred"]* (1 - i/100)).round(2))
    for sl, s, l, c, p in zip(SpotLag_list, Spot_list, Low_list, Close_list, Pred_list_i):
        # Trade on DA market
        if sl <= p:
            Executed_i.append(-s)
            Judge_i.append("True")
            Position_i.append("DA")
        # Trade on Intra markets
        else:
            if p >= l:
                Executed_i.append(-p)
                Judge_i.append("True")
                Position_i.append("Intra")
            else:
                Executed_i.append(-c)
                Judge_i.append("False")           
                Position_i.append("Intra")

    df_buyer_strategy1_ex["Exec_" + str(i) + "%"] = pd.Series(Executed_i)
    df_buyer_strategy1_ex["Judge_" + str(i) + "%"] = pd.Series(Judge_i)
    df_buyer_strategy1_ex["Position_" + str(i) + "%"] = pd.Series(Position_i)
    
df_buyer_strategy1_ex.head()

### Evaluation

In [ ]:
n_splits=100

train_size = df_buyer_strategy1_ex.index[-1]
train_index_list = list(np.linspace(train_size/n_splits, train_size, n_splits, endpoint = True, dtype='int'))
train_index_list

In [ ]:
# This code is used only for confirming the best execution point
BestExec = []

for train_index in train_index_list:
    X_train = df_buyer_strategy1_ex[:train_index]

    cols = []

    # List for the results for evaluation
    PortfolioReturn = []
    StandardDeviation = []
    Max = []
    Min = []
    SharpRatio = []
    InformationRatio=[]

    for i in list(range(0, 51, 1)):
        Return_i = []
        Exec_list_i = list(X_train["Exec_" + str(i) + "%"]) 
        for e in Exec_list_i:
            Return_i.append(e)

        Return_i = pd.Series(Return_i)
        PortfolioReturn.append(Return_i.mean().round(2))
        StandardDeviation.append(Return_i.std().round(2))
        Max.append(Return_i.max().round(2))
        Min.append(Return_i.min().round(2))
        SharpRatio.append((Return_i.mean()/Return_i.std()).round(2))
        InformationRatio.append(round((Return_i - Benchmark_buyer).mean() / (Return_i - Benchmark_buyer).std(), 3))

        # Make columns names
        cols.append(i)

    # Make dataframe for evaluation and  switch columns and row.
    df_buyer_strategy1_ex_eval = pd.DataFrame()

    # Add columns for evaluation metrics
    df_buyer_strategy1_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
    df_buyer_strategy1_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
    df_buyer_strategy1_ex_eval["Max"] = pd.Series(Max)
    df_buyer_strategy1_ex_eval["Min"] = pd.Series(Min)
    df_buyer_strategy1_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
    df_buyer_strategy1_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

    df_buyer_strategy1_ex_eval.index = cols
    df_buyer_strategy1_ex_eval = df_buyer_strategy1_ex_eval.reset_index()
    df_buyer_strategy1_ex_eval = df_buyer_strategy1_ex_eval.rename(columns={"index": "ExecBuffer(%)"})

    Exec = df_buyer_strategy1_ex_eval[df_buyer_strategy1_ex_eval["InformationRatio"] == df_buyer_strategy1_ex_eval["InformationRatio"].max()]
    Exec = -Exec["ExecBuffer(%)"][Exec["StandardDeviation"] == Exec["StandardDeviation"].min()].iloc[0]
    BestExec.append(Exec)

In [ ]:
# The best execution of 100 different periods
BestExec = pd.Series(BestExec)
plt.xlabel('Number of backtest')
plt.ylabel('Execution adjustment (%)')
BestExec.plot();

In [ ]:
Best_i = BestExec.iloc[-1]
Best_i

In [ ]:
cols = []

# List for the results for evaluation
PortfolioReturn = []
StandardDeviation = []
Max = []
Min = []
SharpRatio = []
InformationRatio = []

for i in list(range(0, 51, 1)):
    Return_i = []
    Exec_list_i = list(df_buyer_strategy1_ex["Exec_" + str(i) + "%"]) 
    for e in Exec_list_i:
        Return_i.append(e)

    Return_i = pd.Series(Return_i)
    PortfolioReturn.append(Return_i.mean().round(2))
    StandardDeviation.append(Return_i.std().round(2))
    Max.append(Return_i.max().round(2))
    Min.append(Return_i.min().round(2))
    SharpRatio.append((Return_i.mean()/Return_i.std()).round(2))
    InformationRatio.append(round((Return_i - Benchmark_buyer).mean() / (Return_i - Benchmark_buyer).std(), 3))

    # Make columns names
    cols.append("Exec+" + str(i) + "%")

# Make dataframe for evaluation and  switch columns and row.
df_buyer_strategy1_ex_eval = pd.DataFrame()

# Add columns for evaluation metrics
df_buyer_strategy1_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_buyer_strategy1_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_buyer_strategy1_ex_eval["Max"] = pd.Series(Max)
df_buyer_strategy1_ex_eval["Min"] = pd.Series(Min)
df_buyer_strategy1_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
df_buyer_strategy1_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

df_buyer_strategy1_ex_eval.index = cols

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_buyer_strategy1_ex_eval[df_buyer_strategy1_ex_eval["StandardDeviation"] == df_buyer_strategy1_ex_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].max()]
GMVP

In [ ]:
# The best portfolio
Best = df_buyer_strategy1_ex_eval[df_buyer_strategy1_ex_eval["InformationRatio"] == df_buyer_strategy1_ex_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_buyer_strategy1_ex_eval["StandardDeviation"], df_buyer_strategy1_ex_eval["PortfolioReturn"], c=df_buyer_strategy1_ex_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')

# Global minimum variance portfolio
plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio]")
i = 6
DA_position_i = df_buyer_strategy1_ex["Position_" + str(i) + "%"].value_counts()[0] 
Intra_position_i = df_buyer_strategy1_ex["Position_" + str(i) + "%"].value_counts()[1]
print("Position: DA {}".format(round(DA_position_i / (DA_position_i+Intra_position_i)*100, 2)) + "%")
print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Sharp ratio: {}".format(GMVP["SharpRatio"][0]))
print("Information ratio: {}".format(GMVP["InformationRatio"][0]))

print("[Best Execution Portfolio]")
i = -Best_i
DA_position_i = df_buyer_strategy1_ex["Position_" + str(i) + "%"].value_counts()[0] 
Intra_position_i = df_buyer_strategy1_ex["Position_" + str(i) + "%"].value_counts()[1]
plt.plot(df_buyer_strategy1_ex["Exec_" + str(i) + "%"].std(), df_buyer_strategy1_ex["Exec_" + str(i) + "%"].mean(), "b*", markersize=15.0, label="Best Execution")
print("Position: DA {}".format(round(DA_position_i / (DA_position_i+Intra_position_i)*100, 2)) + "%")
print("Expected return: {}".format(df_buyer_strategy1_ex["Exec_" + str(i) + "%"].mean().round(2)))
print("Standard deviation: {}".format(df_buyer_strategy1_ex["Exec_" + str(i) + "%"].std().round(2)))
print("Sharp ratio: {}".format(round(df_buyer_strategy1_ex["Exec_" + str(i) + "%"].mean() / df_buyer_strategy1_ex["Exec_" + str(i) + "%"].std(),2)))
print("Information ratio: {}".format(round((df_buyer_strategy1_ex["Exec_" + str(i) + "%"] - Benchmark_buyer).mean() / (df_buyer_strategy1_ex["Exec_" + str(i) + "%"] - Benchmark_buyer).std(),2)))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Information Ratio')
plt.legend(loc="upper left");

## Strategy 2 (Trading with prediction after DA)

### Required dataset

In [ ]:
# Need predicted price after DA market and spot price
df_buyer_strategy2 = df_prediction_afterDA.copy()

# Make lists for price information
Spot_list = list(df_buyer_strategy2["Spot"])
Low_list = list(df_buyer_strategy2["Low"])
Close_list = list(df_buyer_strategy2["Close"])
Pred_list_i = list(df_buyer_strategy2["Close_pred"])
# Judge_success = []
                   
# list for executed orders
Executed_i = []
# list for checking the execution results
Judge_i = []    
                   
# Calculate executed price
for l, c, p in zip(Low_list, Close_list, Pred_list_i):     
    # Trade on Intra markets
    if p >= l:
        Executed_i.append(p)
        Judge_i.append("True")
    else:
        Executed_i.append(c)
        Judge_i.append("False")                 

df_buyer_strategy2["ExecutedOrder"] = pd.Series(Executed_i)
df_buyer_strategy2["Judge"] = pd.Series(Judge_i)

In [ ]:
df_buyer_strategy2.head()

### Evaluation

In [ ]:
portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

cols = []

PortfolioReturn = []
StandardDeviation = []
Max = []
Min = []
SharpRatio = []
InformationRatio = []

Spot_list = list(df_buyer_strategy2["Spot"])
Exec_list = list(df_buyer_strategy2["ExecutedOrder"])

for weight in portfolio_weights:
    Return = []
    for spot, intra in zip(Spot_list, Exec_list):
        # DA market 100%
        if weight == 1.0:
            Return.append(-spot*weight)
        # Trade both on DA and Intra day
        else:
            Return.append(-spot*weight + -intra*(1-weight))
    # Set colmun name
    cols.append("DA" + str(weight*100) + "%")

    # Results
    Return = pd.Series(Return)
    PortfolioReturn.append(Return.mean().round(2))
    StandardDeviation.append(Return.std().round(2))
    Max.append(Return.max().round(2))
    Min.append(Return.min().round(2))
    SharpRatio.append((Return.mean()/Return.std()).round(2))
    InformationRatio.append(round((Return - Benchmark_buyer).mean() / (Return - Benchmark_buyer).std(), 3))

# Transpose columns and rows
df_buyer_strategy2_eval = pd.DataFrame()
df_buyer_strategy2_eval = df_buyer_strategy2_eval.T

# Add columns for evaluation metrics
df_buyer_strategy2_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_buyer_strategy2_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_buyer_strategy2_eval["Max"] = pd.Series(Max)
df_buyer_strategy2_eval["Min"] = pd.Series(Min)
df_buyer_strategy2_eval["SharpRatio"] = pd.Series(SharpRatio)
df_buyer_strategy2_eval["InformationRatio"] = pd.Series(InformationRatio)

df_buyer_strategy2_eval.index = cols

In [ ]:
df_buyer_strategy2_eval

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_buyer_strategy2_eval[df_buyer_strategy2_eval["StandardDeviation"] == df_buyer_strategy2_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].min()]
GMVP

In [ ]:
# The Best Execusion
Best = df_buyer_strategy2_eval[df_buyer_strategy2_eval["InformationRatio"] == df_buyer_strategy2_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_buyer_strategy2_eval["StandardDeviation"], df_buyer_strategy2_eval["PortfolioReturn"], c=df_buyer_strategy2_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')

# Global minimum variance portfolio
plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio]")
print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Sharp ratio: {}".format(GMVP["SharpRatio"][0]))
print("Information ratio: {}".format(GMVP["InformationRatio"][0]))

# Best executed portfolio
plt.plot(Best["StandardDeviation"], Best["PortfolioReturn"], "b*", markersize=15.0, label="Best Execution")
print("[Best Execution Portfolio]")
print("Expected return: {}".format(Best["PortfolioReturn"][0]))
print("Standard deviation: {}".format(Best["StandardDeviation"][0]))
print("Sharp ratio: {}".format(Best["SharpRatio"][0]))
print("Information ratio: {}".format(Best["InformationRatio"][0]))

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Information Ratio')
plt.legend(loc="upper right");

## Strategy 2 with execution strategy

### Required dataset

In [ ]:
# Need predicted price after DA market and spot price
df_buyer_strategy2_ex = df_prediction_beforeDA.copy()

# Make lists for price information
Spot_list = list(df_buyer_strategy2_ex["Spot"])
Low_list = list(df_buyer_strategy2_ex["Low"])
Close_list = list(df_buyer_strategy2_ex["Close"])
# Judge_success = []

# Order and execution with execusion range
for i in list(range(0, 51, 1)):
    # list for executed orders
    Executed_i = []
    # list for checking the execution results
    Judge_i = []    
    Position_i = []
    # Derive the predicted price with the range
    Pred_list_i = list((df_buyer_strategy2_ex["Close_pred"]* (1 - i/100)).round(2))
    for l, c, p in zip(Low_list, Close_list, Pred_list_i):     
        # Trade on Intra markets
        if p >= l:
            Executed_i.append(p)
            Judge_i.append("True")
            Position_i.append("DA")
        else:
            Executed_i.append(c)
            Judge_i.append("False") 
            Position_i.append("Intra")
                
    df_buyer_strategy2_ex["Exec_" + str(i) + "%"] = pd.Series(Executed_i)
    df_buyer_strategy2_ex["Judge_" + str(i) + "%"] = pd.Series(Judge_i)
    df_buyer_strategy2_ex["Position_" + str(i) + "%"] = pd.Series(Position_i)
    
df_buyer_strategy2_ex.head()

### Evaluation

- Look for the best execution

In [ ]:
n_splits=100

train_size = df_buyer_strategy2_ex.index[-1]
train_index_list = list(np.linspace(train_size/n_splits, train_size, n_splits, endpoint = True, dtype='int'))
train_index_list

# # Confirming the split logic
# for train_index in train_index_list:
#     # Divide the train/valid set  into 10 folds and pick up it.
#     X_train = df_buyer_strategy2_ex.iloc[:train_index]
#     print("TRAIN:", train_index)

In [ ]:
BestExec = []

for train_index in train_index_list:
    X_train = df_buyer_strategy2_ex[:train_index]

    Spot_list = list(X_train["Spot"])

    portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    execution_buffers = list(range(0, 51, 1))

    cols = []

    PortfolioReturn = []
    StandardDeviation = []
    Max = []
    Min = []
    SharpRatio = []
    InformationRatio = []
    
    for i in execution_buffers:
        Exec_list = list(X_train["Exec_" + str(i) + "%"])
        for weight in portfolio_weights:
            Return = []
            for spot, intra in zip(Spot_list, Exec_list):
                # DA market 100%
                if weight == 1.0:
                    Return.append(-spot*weight)
                # Trade both on DA and Intra day
                else:
                    Return.append(-spot*weight + -intra*(1-weight))
            # Set colmun name for execution buffer
            if weight == 1.0:
                cols.append(str(0))
            else:
                cols.append(str(i))

            # Results
            Return = pd.Series(Return)
            PortfolioReturn.append(Return.mean().round(2))
            StandardDeviation.append(Return.std().round(2))
            Max.append(Return.max().round(2))
            Min.append(Return.min().round(2))
            SharpRatio.append((Return.mean()/Return.std()).round(2))
            InformationRatio.append(round((Return - Benchmark_buyer).mean() / (Return - Benchmark_buyer).std(), 3))

    #df_portfolio_benchの列を設定し、行列を入替えて調整する
    df_buyer_strategy2_ex_eval = pd.DataFrame()
    df_buyer_strategy2_ex_eval = df_buyer_strategy2_ex_eval.T

    #各算出結果をdf_portfolio_benchの列へ追加する
    df_buyer_strategy2_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
    df_buyer_strategy2_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
    df_buyer_strategy2_ex_eval["Max"] = pd.Series(Max)
    df_buyer_strategy2_ex_eval["Min"] = pd.Series(Min)
    df_buyer_strategy2_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
    df_buyer_strategy2_ex_eval["InformationRatio"] = pd.Series(InformationRatio)
    
    df_buyer_strategy2_ex_eval.index = cols
    df_buyer_strategy2_ex_eval =  df_buyer_strategy2_ex_eval.reset_index()
    df_buyer_strategy2_ex_eval = df_buyer_strategy2_ex_eval.rename(columns={"index": "ExecBuffer(%)"})
    df_buyer_strategy2_ex_eval["ExecBuffer(%)"] = df_buyer_strategy2_ex_eval["ExecBuffer(%)"].astype(int)

    Exec = df_buyer_strategy2_ex_eval[df_buyer_strategy2_ex_eval["InformationRatio"] == df_buyer_strategy2_ex_eval["InformationRatio"].max()]
    Exec = -Exec["ExecBuffer(%)"][Exec["StandardDeviation"] == Exec["StandardDeviation"].min()].iloc[0]
    BestExec.append(Exec)

In [ ]:
# BestExec = pd.Series(BestExec)
BestExec = pd.Series(BestExec).astype(int)
plt.xlabel('Number of backtest')
plt.ylabel('Execution adjustment (%)')
BestExec.plot();

In [ ]:
Best_i = BestExec.iloc[-1]
Best_i

- Plot all the possible portfolios

In [ ]:
Spot_list = list(df_buyer_strategy2_ex["Spot"])

portfolio_weights= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
execution_buffers = list(range(0, 51, 1))

cols = []

PortfolioReturn = []
StandardDeviation = []
Max = []
Min = []
SharpRatio = []
InformationRatio = []

for i in execution_buffers:
    Exec_list = list(df_buyer_strategy2_ex["Exec_" + str(i) + "%"])
    for weight in portfolio_weights:
        Return = []
        for spot, intra in zip(Spot_list, Exec_list):
            # DA market 100%
            if weight == 1.0:
                Return.append(-spot*weight)
            # Trade both on DA and Intra day
            else:
                Return.append(-spot*weight + -intra*(1-weight))
        # Set colmun name for execution buffer
        if weight == 1.0:
            cols.append("DA100%")
        else:
            cols.append("DA" + str(weight*100) + "%_Exec+" + str(i) + "%")

        # Results
        Return = pd.Series(Return)
        PortfolioReturn.append(Return.mean().round(2))
        StandardDeviation.append(Return.std().round(2))
        Max.append(Return.max().round(2))
        Min.append(Return.min().round(2))
        SharpRatio.append((Return.mean()/Return.std()).round(2))
        InformationRatio.append(((Return - Benchmark_buyer).mean()/(Return - Benchmark_buyer).std()).round(3))

#df_portfolio_benchの列を設定し、行列を入替えて調整する
df_buyer_strategy2_ex_eval = pd.DataFrame()
df_buyer_strategy2_ex_eval = df_buyer_strategy2_ex_eval.T

#各算出結果をdf_portfolio_benchの列へ追加する
df_buyer_strategy2_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
df_buyer_strategy2_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
df_buyer_strategy2_ex_eval["Max"] = pd.Series(Max)
df_buyer_strategy2_ex_eval["Min"] = pd.Series(Min)
df_buyer_strategy2_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
df_buyer_strategy2_ex_eval["InformationRatio"] = pd.Series(InformationRatio)

df_buyer_strategy2_ex_eval.index = cols

In [ ]:
df_buyer_strategy2_ex_eval

In [ ]:
# Pick up the portfolios on the global minimum variance portfolio
std_min = df_buyer_strategy2_ex_eval[df_buyer_strategy2_ex_eval["StandardDeviation"] == df_buyer_strategy2_ex_eval["StandardDeviation"].min()]
GMVP = std_min[std_min["PortfolioReturn"] == std_min["PortfolioReturn"].min()]
GMVP

In [ ]:
Best = df_buyer_strategy2_ex_eval[df_buyer_strategy2_ex_eval["InformationRatio"] == df_buyer_strategy2_ex_eval["InformationRatio"].max()]
Best = Best[Best["StandardDeviation"] == Best["StandardDeviation"].min()]
Best

In [ ]:
plt.figure(figsize=(10,6))

# Scatter plot all the possible portfolios
plt.scatter(df_buyer_strategy2_ex_eval["StandardDeviation"], df_buyer_strategy2_ex_eval["PortfolioReturn"], c=df_buyer_strategy2_ex_eval["InformationRatio"], marker='.', alpha=0.8, cmap='coolwarm')


# Global minimum variance portfolio
plt.plot(GMVP["StandardDeviation"], GMVP["PortfolioReturn"], 'r*', markersize=15.0, label="GMVP")
print("[Global Minimum Variance Portfolio: " + str(GMVP.index[0]) + "]")

print("Expected return: {}".format(GMVP["PortfolioReturn"][0]))
print("Standard deviation: {}".format(GMVP["StandardDeviation"][0]))
print("Information Ratio: {}".format(GMVP["InformationRatio"][0]))

# The Best Execusion
DA_weight = 0
i = -Best_i
Best_Portfolio = (DA_weight * -df_buyer_strategy2_ex["Spot"]) + ((1 - DA_weight) * -df_buyer_strategy2_ex["Exec_" + str(i) + "%"])
plt.plot(Best_Portfolio.std(), Best_Portfolio.mean(), "b*", markersize=15.0, label="Best Execution")
print("[Best Execution Portfolio: " + str(DA_weight) + "%_Exec+" + str(i) + "%]")
print("Expected return: {}".format(Best_Portfolio.mean().round(2)))
print("Standard deviation: {}".format(Best_Portfolio.std().round(2)))
print("Sharp ratio: {}".format(round(Best_Portfolio.mean() / Best_Portfolio.std(), 2)))
print("Information ratio: {}".format(round((Best_Portfolio - Benchmark_buyer).mean() / (Best_Portfolio - Benchmark_buyer).std(), 3)))

plt.title('Strategic portfolios with excution adjustment for buyers')
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Information Ratio')
plt.legend(loc="upper left");

In [ ]:
Best_Portfolio

# For Trader (Virtual bidding strategy)

[Purpose]
- To confirm the market efficiency of the JEPX 
- To examine virtual bidding as a potential solution for improving liquidity of the DA and the intraday market on the JEPX

[Basic logic for virtual bidding] <p>
1) Make long or short positions on the DA market <p>
2) Settle all the positions by counter trading on the intraday market on the day <p>
    - Spot > Close: Entry with sell-in on the DA  and buy-out on the intraday <p>
    - Spot < Close: Entry with buy-in on the DA and sell-out on the intraday

## Trading without execution strategy

In [ ]:
df_trader = df_prediction_beforeDA.copy()
df_trader["Date"] = pd.to_datetime(df_trader["Date"])
df_trader["DateTime"] = pd.to_datetime(df_trader["DateTime"])
df_trader.head()

### Evaluation

In [ ]:
# For execution
LaggedSpot_list = list(df_trader["Spot_1daylag"])
Pred_list = list(df_trader["Close_pred"])
Close_list = list(df_trader["Close"])
High_list = list(df_trader["High"])
Low_list = list(df_trader["Low"])

# For evaluation
Spot_list = list(df_trader["Spot"])

# Lists for results
cols = []
Return = [] 
PortfolioReturn = []
StandardDeviation = []
Max = []
Min = []
SharpRatio = []

# Main logic
for spot, lag_spot, pred, high, low, close in zip(Spot_list,LaggedSpot_list, Pred_list, High_list, Low_list, Close_list):
    #　Buy-in, Sell-out　※ Decision based on lag_spot and pred
    if lag_spot < pred:
        #Return ※ Evaluation based on spot and pred/close
        # Succeed
        if pred < high:
            Return.append(round(pred - spot, 4))
        # Fail
        else:
            Return.append(round(close - spot, 4))
            
    # Sell-in, Buy-out ※ Decision based on lag_spot and pred
    elif lag_spot > pred:
        #Return ※ Evaluation based on spot and pred/close
        # Succeed
        if pred > low:
            Return.append(round(spot - pred, 4))
        # Fail
        else:
            Return.append(round(spot - close, 4))
    # lag_spot is the same as pred --> No trade
    else:
        Return.append(0)

df_trader_eval = pd.DataFrame()
df_trader_eval["DateTime"] = df_trader["DateTime"]
df_trader_eval["HH"] = df_trader["HH"]
df_trader_eval["Date"] = df_trader["Date"]
df_trader_eval["Return"] = pd.Series(Return)

In [ ]:
df_trader_eval.head()

In [ ]:
# Make a pivot table for results
df_trader_portfolio_table = pd.DataFrame(df_trader_eval.pivot(index='Date', columns='HH', values='Return'))
df_trader_portfolio_table["Expected_Return"] = df_trader_portfolio_table.loc[: , 0:48].mean(axis=1).round(2)
df_trader_portfolio_table["SharpRatio"] = (df_trader_portfolio_table.loc[: , 1:48].mean(axis=1)/df_trader_portfolio_table.loc[: , 1:48].std(axis=1)).round(2)

# Make a new table for performance graph
risk = pd.DataFrame()
risk["Expected_Return"] = df_trader_portfolio_table.loc[: , 1:48].mean(axis=1).round(2)
risk["equity"] = risk['Expected_Return'].cumsum()
risk["cummax"] = risk["equity"].cummax()
risk["drawdown"] = -(risk["cummax"] -risk["equity"])

risk.head()

In [ ]:
# Set max_drawdown
max_drawdown = risk["drawdown"].min()

# Set timestamp of max_drawdown
t_max = pd.to_datetime(risk["drawdown"].idxmin())

# Plot the performance
fig, ax = plt.subplots(1, figsize=(15,8))
# plt.title('Cumulative return through trading based on the difference of price on DA and Intraday')

risk["equity"].plot(figsize=(10, 6), label="Cum_Return")

plt.text(t_max, 0, 'Max_D', rotation=0, color="r")

print("Expected_Return: " + str(risk["Expected_Return"].mean().round(2)))
print("StandardDeviation: " + str(risk["Expected_Return"].std().round(2)))
print("SharpRatio: " + str(round(risk["Expected_Return"].mean() /  risk["Expected_Return"].std(),2)))
print("Cum_Return: " + str(risk["equity"][-1:][0].round(2)))
print("Drawdown: " + str(risk["drawdown"].min().round(2)))

ax.set(xlabel="Date", ylabel="Cumulative return(Yen/kWh)")
plt.axvline(t_max, c="r", alpha=0.5)
plt.grid()
plt.legend();

In [ ]:
# Distribution of expected return
plt.figure(figsize=(5, 3))

plt.hist(risk['Expected_Return'], bins=100);

In [ ]:
# Calculate VaR
import scipy.stats as scs

equity = 10000000

percs = np.array([1., 5.0, 10.0])
risk["returns"] = np.log(risk["equity"] / risk["equity"].shift(1))
VaR = scs.scoreatpercentile(equity * risk["returns"], percs)
def print_var():
    print('%16s %16s' % ('Confidence Level', 'Value-at-Risk'))
    print(43 * '-')
    for pair in zip(percs, VaR):
        print('%16.0f %16.0f' % (100 - pair[0], -pair[1]))
        
print_var()

In [ ]:
# Plot each 48 item separately
fig, ax = plt.subplots(1, figsize=(15,8))
# plt.title('Cumulative return through trading based on the difference of price on DA and Intraday')

# Lists for results
Equity = []
ExpectedReturn = []
Max = []
Min = []
StandardDeviation = []
SharpRatio= []
CumMax = []
MaxDrawdown = []

for HH in list(range(1, 49, 1)):
    #　makng a pivot table
    df_trader_portfolio_table = pd.DataFrame(df_trader_eval.pivot(index='Date', columns='HH', values='Return'))
    
    # For plot and for calculating drawdown
    equity = df_trader_portfolio_table[HH].cumsum().round(2)
    # Plot the performance of each HH spot
    equity.plot(label="HH" + str(HH))
    
    # Keep in a list for calculating drawdown
    Equity.append(equity)
    cummax = equity.cummax()
    drawdown = -(cummax - equity)

    # Calculation for each evaluation on each HH slot
    HH_Return = df_trader_portfolio_table[HH].mean().round(2)
    HH_Std = df_trader_portfolio_table[HH].std().round(2)
    ExpectedReturn.append(HH_Return)
    StandardDeviation.append(HH_Std)
    Max.append(df_trader_portfolio_table[HH].max().round(2))
    Min.append(df_trader_portfolio_table[HH].min().round(2))
    SharpRatio.append((HH_Return / HH_Std).round(2))
    CumMax.append(cummax.max().round(2))
    MaxDrawdown.append(drawdown.min().round(2))

ax.set(xlabel="Date", ylabel="Cumulative return(Yen/kWh)")
plt.grid()
plt.legend(loc="upper left", ncol=6);

In [ ]:
# Make a table for evaluation metrics for all items
df_trader_HH_Eval = pd.DataFrame()
df_trader_HH_Eval["HH"] =  pd.Series(list(range(1,49,1)))
df_trader_HH_Eval["ExpectedReturn"]  = pd.Series(ExpectedReturn)
df_trader_HH_Eval["Max"]  = pd.Series(Max)
df_trader_HH_Eval["Min"]  = pd.Series(Min)
df_trader_HH_Eval["StandardDeviation"]  = pd.Series(StandardDeviation)
df_trader_HH_Eval["SharpRatio"]  = pd.Series(SharpRatio)
df_trader_HH_Eval["CumMax"] = pd.Series(CumMax)
df_trader_HH_Eval["MaxDrawdown"]  = pd.Series(MaxDrawdown)

In [ ]:
df_trader_HH_Eval[df_trader_HH_Eval["CumMax"] == df_trader_HH_Eval["CumMax"].max()]

HH46 --> 22:30

In [ ]:
df_trader_HH_Eval[df_trader_HH_Eval["SharpRatio"] == df_trader_HH_Eval["SharpRatio"].max()]

HH3 --> 1:00

In [ ]:
df_trader_HH_Eval[df_trader_HH_Eval["CumMax"] == df_trader_HH_Eval["CumMax"].min()]

HH18 --> 8:30

In [ ]:
df_trader_HH_Eval[df_trader_HH_Eval["MaxDrawdown"] == df_trader_HH_Eval["MaxDrawdown"].max()]

HH6 --> 2:30

In [ ]:
# Distribution of expected return
fig, ax = plt.subplots(1, figsize=(5,3))
plt.hist(df_trader_portfolio_table[6], bins=100)
plt.ylim(0, 500)
ax.set(xlabel="ExpectedReturn", ylabel="Frequency")
plt.grid();

In [ ]:
df_trader_HH_Eval[df_trader_HH_Eval["MaxDrawdown"] == df_trader_HH_Eval["MaxDrawdown"].min()]

HH34 --> 16:30

In [ ]:
# Distribution of expected return
fig, ax = plt.subplots(1, figsize=(5,3))
df_trader_portfolio_table[34].hist(bins=100)
plt.ylim(0, 500)
ax.set(xlabel="ExpectedReturn", ylabel="Frequency");

## Trading with execution strategy

### Required dataset

In [ ]:
Spot_list = list(df_trader["Spot"])
LaggedSpot_list = list(df_trader["Spot_1daylag"])
Close_list = list(df_trader["Close"])
High_list = list(df_trader["High"])
Low_list = list(df_trader["Low"])

cols = []

df_trader_ex = pd.DataFrame()
df_trader_ex["DateTime"] = df_trader["DateTime"]
df_trader_ex["HH"] = df_trader["HH"]
df_trader_ex["Date"] = df_trader["Date"]

# Just for check
df_trader_ex["Spot"] = df_trader["Spot"]
df_trader_ex["Close_pred"] = df_trader["Close_pred"]
df_trader_ex["Close"] = df_trader["Close"]

for i in list(range(-30, 31, 1)):
    Return = [] 
    Judge = []      
    Pred_list = list(round(df_trader_ex["Close_pred"]  * (1+ i/100),2))
    for spot, lag_spot, pred_ex, high, low, close in zip(Spot_list, LaggedSpot_list, Pred_list, High_list, Low_list, Close_list):
        #　Buy-in, Sell-out　※Decision based on lag_spot and pred
        if lag_spot < pred_ex:
            #※ Evaluation based on spot and pred/close
            # Success of close the position
            if pred_ex <= high:
                Return.append(pred_ex - spot)
                Judge.append("True")                
            # Failure of close the position
            else:
                Return.append(close - spot)
                Judge.append("False")                

        # Sell-in, Buy-out ※ Decision based on lag_spot and pred
        elif lag_spot > pred_ex:
            #※ Evaluation based on spot and pred/close
            # Success of close the position
            if pred_ex >= low:
                Return.append(spot - pred_ex)
                Judge.append("True")                
            # Failure of close the position
            else:
                Return.append(spot - close)
                Judge.append("False")                
        #No trade
        else:
            Return.append(0)
            Judge.append("None")                

    df_trader_ex["Return_exec" + str(i) + "%"] = pd.Series(Return)
    df_trader_ex["Judge_exec" + str(i) + "%"] = pd.Series(Judge)

In [ ]:
df_trader_ex

### Evaluation

In [ ]:
n_splits=100

train_size = df_trader_ex.index[-1]
train_index_list = list(np.linspace(train_size/n_splits, train_size, n_splits, endpoint = True, dtype='int'))
train_index_list

# Confirming the split logic
for train_index in train_index_list:
    # Divide the train/valid set  into 10 folds and pick up it.
    X_train = df_trader_ex.iloc[:train_index]
    print("TRAIN:", train_index)

In [ ]:
BestExec = []

for train_index in train_index_list:
    X_train = df_trader_ex[:train_index]
    
    cols = []

    # List for the results for evaluation
    PortfolioReturn = []
    StandardDeviation = []
    Max = []
    Min = []
    SharpRatio = []

    for i in list(range(-30, 31, 1)):
        Return_i = list(X_train["Return_exec" + str(i) + "%"]) 
        
        Return_i = pd.Series(Return_i)
        PortfolioReturn.append(Return_i.mean().round(2))
        StandardDeviation.append(Return_i.std().round(2))
        Max.append(Return_i.max().round(2))
        Min.append(Return_i.min().round(2))
        SharpRatio.append((Return_i.mean()/Return_i.std()).round(2))
        # Make columns names
        cols.append(i)

    # Make dataframe for evaluation and  switch columns and row.
    df_trader_ex_eval = pd.DataFrame()
    # df_trader_ex_eval = df_trader_ex_eval.T

    # Add columns for evaluation metrics 
    df_trader_ex_eval["PortfolioReturn"] = pd.Series(PortfolioReturn)
    df_trader_ex_eval["StandardDeviation"] = pd.Series(StandardDeviation)
    df_trader_ex_eval["Max"] = pd.Series(Max)
    df_trader_ex_eval["Min"] = pd.Series(Min)
    df_trader_ex_eval["SharpRatio"] = pd.Series(SharpRatio)
    df_trader_ex_eval.index = cols
    df_trader_ex_eval =  df_trader_ex_eval.reset_index()
    df_trader_ex_eval = df_trader_ex_eval.rename(columns={"index": "ExecBuffer(%)"})
    df_trader_ex_eval["ExecBuffer(%)"] = df_trader_ex_eval["ExecBuffer(%)"].astype(int)

    BestExec.append(df_trader_ex_eval["ExecBuffer(%)"][df_trader_ex_eval["SharpRatio"] == df_trader_ex_eval["SharpRatio"].max()].max())

In [ ]:
# The best execution of 100 different periods
BestExec = pd.Series(BestExec)
plt.xlabel('Number of backtest')
plt.ylabel('Execution adjustment (%)')
BestExec.plot();

In [ ]:
BestExec.tail()

In [ ]:
BestExec[-1:]

In [ ]:
i = int(BestExec[-1:])

# Pivot table for evaluation metrics
df_trader_ex_table = pd.DataFrame(df_trader_ex.pivot(index='Date', columns='HH', values='Return_exec' + str(i)  +"%"))
df_trader_ex_table["Expected_Return"] = df_trader_ex_table.loc[: , 0:48].mean(axis=1).round(2)
df_trader_ex_table["StandardDeviation"] = df_trader_ex_table.loc[: , 1:48].std(axis=1).round(2)
df_trader_ex_table["Max"] = df_trader_ex_table.loc[: , 1:48].max(axis=1).round(2)
df_trader_ex_table["Min"] = df_trader_ex_table.loc[: , 1:48].min(axis=1).round(2)
df_trader_ex_table["SharpRatio"] = (df_trader_ex_table.loc[: , 1:48].mean(axis=1)/df_trader_ex_table.loc[: , 1:48].std(axis=1)).round(2)

# Make a new table for performance graph
risk_ex = pd.DataFrame()
risk_ex["Expected_Return"] = df_trader_ex_table.loc[: , 1:48].mean(axis=1).round(2)
risk_ex["equity"] = risk_ex['Expected_Return'].cumsum()
risk_ex["cummax"] = risk_ex["equity"].cummax()
risk_ex["drawdown"] = -(risk_ex["cummax"] - risk_ex["equity"])

# Pick up max_drawdown
max_drawdown = risk_ex["drawdown"].min()

# Set timestamp of max_drawdown
t_max = pd.to_datetime(risk_ex["drawdown"].idxmin())

fig, ax = plt.subplots(1, figsize=(15,8))
# plt.title('Cumulative return through trading based on the difference of price on DA and Intraday')

risk_ex["equity"].plot(figsize=(10, 6))

plt.text(t_max, 0, 'Max_drawdown', rotation=0, color="r")

print("Expected_Return: " + str(risk_ex["Expected_Return"].mean().round(2)))
print("StandardDeviation: " + str(risk_ex["Expected_Return"].std().round(2)))
print("SharpRatio: " + str(round(risk_ex["Expected_Return"].mean() /  risk_ex["Expected_Return"].std(), 3)))
print("Cumulative_Return: " + str((risk_ex["cummax"].max().round(2))))
print("Drawdown: " + str(risk_ex["drawdown"].min().round(2)))

ax.set(xlabel="Date", ylabel="Cumulative return(Yen/kWh)")
plt.axvline(t_max, c="r", alpha=0.5)
plt.grid()
plt.legend();

In [ ]:
# Calculate VaR

equity = 10000000

percs = np.array([1., 5.0, 10.0])
risk_ex["returns"] = np.log(risk_ex["equity"] / risk_ex["equity"].shift(1))
VaR = scs.scoreatpercentile(equity * risk_ex["returns"], percs)
def print_var():
    print('%16s %16s' % ('Confidence Level', 'Value-at-Risk'))
    print(43 * '-')
    for pair in zip(percs, VaR):
        print('%16.0f %16.0f' % (100 - pair[0], -pair[1]))
        
print_var()

In [ ]:
# Plot each 48 item separately
fig, ax = plt.subplots(1, figsize=(15,8))
plt.title('Cumulative return of virtual bidding strategy for each item after execution adjustment')

# Lists for results
Equity = []
ExpectedReturn = []
Max = []
Min = []
StandardDeviation = []
SharpRatio= []
CumMax = []
MaxDrawdown = []

# Execution baffer　(%)
i = -23

for HH in list(range(1, 49, 1)):
    # Plot table based on Return
    df_trader_portfolio_ex_table = pd.DataFrame(df_trader_ex.pivot(index='Date', columns='HH', values='Return_exec' + str(i)  +"%"))
    
    # For plot and for calculating drawdown
    equity = df_trader_portfolio_ex_table[HH].cumsum().round(2)
    # Plot the performance of each HH spot
    equity.plot(label="HH" + str(HH))
    
    # Keep in a list for calculating drawdown
    Equity.append(equity)
    cummax = equity.cummax()
    drawdown = -(cummax - equity)

    # Calculation for each evaluation on each HH slot
    HH_Return = df_trader_portfolio_ex_table[HH].mean().round(2)
    HH_Std = df_trader_portfolio_ex_table[HH].std().round(2)
    ExpectedReturn.append(HH_Return)
    StandardDeviation.append(HH_Std)
    Max.append(df_trader_portfolio_ex_table[HH].max().round(2))
    Min.append(df_trader_portfolio_ex_table[HH].min().round(2))
    SharpRatio.append((HH_Return / HH_Std).round(2))
    CumMax.append(cummax.max().round(2))
    MaxDrawdown.append(drawdown.min().round(2))

ax.set(xlabel="Date", ylabel="Cumulative return(Yen/kWh)")
plt.grid()
plt.legend(loc="upper left", ncol=6);

In [ ]:
df_trader_HH_ex_Eval = pd.DataFrame()
df_trader_HH_ex_Eval["HH"] =  pd.Series(list(range(1,49,1)))

df_trader_HH_ex_Eval["ExpectedReturn"]  = pd.Series(ExpectedReturn)
df_trader_HH_ex_Eval["Max"]  = pd.Series(Max)
df_trader_HH_ex_Eval["Min"]  = pd.Series(Min)
df_trader_HH_ex_Eval["StandardDeviation"]  = pd.Series(StandardDeviation)
df_trader_HH_ex_Eval["SharpRatio"]  = pd.Series(SharpRatio)
df_trader_HH_ex_Eval["CumMax"] = pd.Series(CumMax)
df_trader_HH_ex_Eval["MaxDrawdown"]  = pd.Series(MaxDrawdown)

In [ ]:
df_trader_HH_ex_Eval[df_trader_HH_ex_Eval["CumMax"] == df_trader_HH_ex_Eval["CumMax"].max()]

HH31 --> 15:00

In [ ]:
df_trader_HH_ex_Eval[df_trader_HH_ex_Eval["CumMax"] == df_trader_HH_ex_Eval["CumMax"].min()]

HH8 --> 3:30

In [ ]:
df_trader_HH_ex_Eval[df_trader_HH_ex_Eval["MaxDrawdown"] == df_trader_HH_ex_Eval["MaxDrawdown"].max()]

HH48 --> 23:30

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,3))
df_trader_portfolio_ex_table[48].hist(bins=100)
plt.ylim(0, 500)
ax.set(xlabel="ExpectedReturn", ylabel="Frequency");

In [ ]:
df_trader_HH_ex_Eval[df_trader_HH_ex_Eval["MaxDrawdown"] == df_trader_HH_ex_Eval["MaxDrawdown"].min()]

HH39 --> 19:00

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,3))
df_trader_portfolio_ex_table[39].hist(bins=100)
plt.ylim(0, 500)
ax.set(xlabel="ExpectedReturn", ylabel="Frequency");